Step 0:
Data and goal
Kaggle project, maximize roc curve (but use gini for evaluation)


step1: research
Before I type a line of code, I need to understand my problem, what approaches other data scientists have used. 
Reading papers, etc. 
Because kaggle is a collaborative community,  I'm focusing my research on other posted kaggle kernels. 

This is a strange problem: To protect customer privacy all data has been anonomized, so there's no way to add domain knowledge in feature engineering. 
As a kaggle competition, there are none of the 'real world' concerns about model implementation, computational costs.
This is also a highly imbalanced problem (1s are much more common than 0s).
I'm going to use this as an example of the standard kaggle workflow: 
Stack  a few different types of boosted tries (extensively hyperparameter tuned)


https://www.kaggle.com/pavetr/stacking-lb-0-285  shows a full stack of multiple models, this is the 'kaggle' approach in a nutshell in all of it's glory (don't feature engineer, make another model and stack them)

https://www.kaggle.com/ogrellier/feature-selection-target-permutations talks about feature permutations (to help with feature engineering)

https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/  (how to hyperparamter tune xgboost)

https://www.kaggle.com/arthurtok/interactive-porto-insights-a-plot-ly-tutorial  EDA tutorial

https://codesachin.wordpress.com/2016/06/25/non-mathematical-feature-engineering-techniques-for-data-science/


My procedure: Use three models (Xgboost, LGBM, Catboost), and use a simple average. 
As a bonus test, I'll make  an LR  that takes as input the outputs  from the three models and returns an output that is the target. 
    

Step 2: Data loading, library loading

In [1]:
import os
from multiprocessing import set_start_method
set_start_method("forkserver")
os.environ["OMP_NUM_THREADS"] = "26" # I run this on  a dual e-52670 (16 core, 32 thread) box. This gives the os 6 threads to play with. 

%env JOBLIB_TEMP_FOLDER=/opt/data/tmp

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from numba import jit
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import warnings
from collections import Counter
from sklearn.feature_selection import mutual_info_classif
warnings.filterwarnings('ignore')
from sklearn.grid_search import GridSearchCV 
from sklearn import metrics
from catboost import CatBoostClassifier
%matplotlib inline
%matplotlib inline

env: JOBLIB_TEMP_FOLDER=/opt/data/tmp


/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from catboost import CatBoostClassifier
!pip install plotly

In [3]:
!pip install seaborn

In [4]:
!pip install xgboost

In [5]:
!pip install lightgbm

In [6]:
!pip install catboost

In [7]:
!pip install numba

In [8]:
#Define Gini metric, this competition uses an uncommmon evaluation metric, so we should define it

@jit  # for more info please visit https://numba.pydata.org/
def eval_gini(y_true, y_prob):
    """
    Original author CMPM 
    https://www.kaggle.com/cpmpml/extremely-fast-gini-computation
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

Step 3: EDA
I'm going to do something a little different from "standard" eda. I'm merely going to focus on feature importance and use that to build some new features, as well as cull features that aren't adding information. 

In [9]:
train = pd.read_csv('train.csv')
# from https://www.kaggle.com/arthurtok/interactive-porto-insights-a-plot-ly-tutorial we can see a method of feature improtance using random forests. 

In [10]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=150, max_depth=8, min_samples_leaf=4, max_features=0.2, n_jobs=-1, random_state=0)
rf.fit(train.drop(['id', 'target'],axis=1), train.target)
features = train.drop(['id', 'target'],axis=1).columns.values
print("----- Training Done -----")

----- Training Done -----


In [11]:
# Scatter plot 
trace = go.Scatter(
    y = rf.feature_importances_,
    x = features,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 13,
        #size= rf.feature_importances_,
        #color = np.random.randn(500), #set color equal to a variable
        color = rf.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = features
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
     xaxis= dict(
         ticklen= 5,
         showgrid=False,
        zeroline=False,
        showline=False
     ),
    yaxis=dict(
        title= 'Feature Importance',
        showgrid=False,
        zeroline=False,
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

In [12]:
x, y = (list(x) for x in zip(*sorted(zip(rf.feature_importances_, features), 
                                                            reverse = False)))
trace2 = go.Bar(
    x=x ,
    y=y,
    marker=dict(
        color=x,
        colorscale = 'Viridis',
        reversescale = True
    ),
    name='Random Forest Feature importance',
    orientation='h',
)

layout = dict(
    title='Barplot of Feature importances',
     width = 900, height = 2000,
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
#         domain=[0, 0.85],
    ))

fig1 = go.Figure(data=[trace2])
fig1['layout'].update(layout)
py.iplot(fig1, filename='plots')

In [13]:
# from https://www.kaggle.com/ogrellier/feature-selection-target-permutations there is alternative eda to check on feature importance
trn_df = pd.read_csv("train.csv", index_col=0)
target = trn_df.target
del trn_df["target"]

In [14]:
clf = LGBMClassifier(boosting_type="rf",
                     num_leaves=1024,
                     max_depth=6,
                     n_estimators=500, 
                     subsample=.632,
                     colsample_bytree=.5,
                     n_jobs=24)

In [15]:
n_splits = 2
n_runs = 5
imp_df = np.zeros((len(trn_df.columns), n_splits * n_runs))
np.random.seed(9385610)
idx = np.arange(len(target))
for run in range(n_runs):
    # Shuffle target
    np.random.shuffle(idx)
    perm_target = target.iloc[idx]
    # Create a new split
    folds = StratifiedKFold(n_splits, True, None)
    oof = np.empty(len(trn_df))
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(perm_target, perm_target)):
        trn_dat, trn_tgt = trn_df.iloc[trn_idx], perm_target.iloc[trn_idx]
        val_dat, val_tgt = trn_df.iloc[val_idx], perm_target.iloc[val_idx]
        # Train classifier
        clf.fit(trn_dat, trn_tgt)
        # Keep feature importances for this fold and run
        imp_df[:, n_splits * run + fold_] = clf.feature_importances_
        # Update OOF for gini score display
        oof[val_idx] = clf.predict_proba(val_dat)[:, 1]
        
    print("Run %2d OOF score : %.6f" % (run, eval_gini(perm_target, oof)))

Run  0 OOF score : 0.009595
Run  1 OOF score : 0.002024
Run  2 OOF score : -0.005417
Run  3 OOF score : 0.000546
Run  4 OOF score : -0.001377


In [16]:
bench_imp_df = np.zeros((len(trn_df.columns), n_splits * n_runs))
for run in range(n_runs):
    # Shuffle target AND dataset
    np.random.shuffle(idx)
    perm_target = target.iloc[idx]
    perm_data = trn_df.iloc[idx]
    
    # Create a new split
    folds = StratifiedKFold(n_splits, True, None)
    oof = np.empty(len(trn_df))
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(perm_target, perm_target)):
        trn_dat, trn_tgt = perm_data.iloc[trn_idx], perm_target.iloc[trn_idx]
        val_dat, val_tgt = perm_data.iloc[val_idx], perm_target.iloc[val_idx]
        # Train classifier
        clf.fit(trn_dat, trn_tgt)
        # Keep feature importances for this fold and run
        bench_imp_df[:, n_splits * run + fold_] = clf.feature_importances_
        # Update OOF for gini score display
        oof[val_idx] = clf.predict_proba(val_dat)[:, 1]
        
    print("Run %2d OOF score : %.6f" % (run, eval_gini(perm_target, oof)))

Run  0 OOF score : 0.257034
Run  1 OOF score : 0.258336
Run  2 OOF score : 0.256461
Run  3 OOF score : 0.254122
Run  4 OOF score : 0.255370


In [17]:
bench_mean = bench_imp_df.mean(axis=1)
perm_mean = imp_df.mean(axis=1)

values = []
for i, f in enumerate(trn_df.columns):
    values.append((f, bench_mean[i], perm_mean[i], bench_mean[i] / perm_mean[i]))

print("%-20s | benchmark | permutation | Ratio" % "Feature")
values = sorted(values, key=lambda x: x[3])
for f, b, p, r in values[::-1]:
    print("%-20s |   %7.1f |     %7.1f |   %7.1f" 
          % (f, b, p, r))

Feature              | benchmark | permutation | Ratio
ps_ind_17_bin        |     617.7 |        96.6 |       6.4
ps_car_07_cat        |     835.1 |       143.1 |       5.8
ps_ind_05_cat        |    1529.9 |       266.6 |       5.7
ps_ind_07_bin        |     524.3 |       113.8 |       4.6
ps_ind_16_bin        |     515.0 |       113.2 |       4.5
ps_car_03_cat        |     751.0 |       179.4 |       4.2
ps_ind_06_bin        |     385.2 |       110.1 |       3.5
ps_car_02_cat        |     225.3 |        83.0 |       2.7
ps_car_04_cat        |     403.0 |       183.2 |       2.2
ps_car_08_cat        |     155.8 |        71.3 |       2.2
ps_ind_03            |    1476.6 |       733.7 |       2.0
ps_ind_08_bin        |     177.7 |        93.6 |       1.9
ps_car_01_cat        |     982.8 |       552.3 |       1.8
ps_reg_02            |    1226.7 |       726.7 |       1.7
ps_ind_15            |    1315.3 |       794.6 |       1.7
ps_ind_01            |     793.8 |       482.7 |       1.6
p

Step 4: test-train splitter:
For a project list this, it's better to do your own train test split before a 5 fold CV: 5 fold CV grid search with some of these models (like xgboost), is computationally very expensive. 


In [18]:

test = pd.read_csv('test.csv')
ssX = train.drop(['id','target'],axis=1)
ssy = train['target']
train_x_fe=ssX
train_y=ssy

In [19]:
train_train_x, train_test_x, train_train_y, train_test_y = train_test_split(ssX, ssy, test_size=0.2, stratify=ssy, random_state=42)

Step 4: Feature Extraction


In [20]:
# This is a process from a kaggle kernel, to explain, we are using some randomized attemts at classifiers to find what are the most import features.
# we can then create new features by combining two of the most important features. 


# feature engineering workflow:
ps_ind_13_bin  
ps_ind_12_bin  
ps_ind_11_bin        
ps_ind_10_bin
ps_ind_calc_20
ps_ind_cal_15
produce so little value the features should be dropped (any feature not contributing is liekly random noise, and gives the model a chance to overfit)

So the next question is how to engineer new  features? 
the binaries of 17,16,7,6 all add value, so we'll add a feature that is the sum of those 4 columns. 

PS car_13 and ps_reg 03 seem to be important, so we'll add them as a feature, and multiply them as a feature. 

car 13 squared, 

reg 03 squared

car 14, car 12 , ind 15 sum

car 14, car 12 ind 15 multiplied as a feature 


In [21]:
# dropping nonvalued data
test_fe = test.drop(['ps_ind_13_bin','ps_ind_12_bin', 'ps_ind_11_bin','ps_ind_10_bin','ps_calc_20_bin'],axis=1)

In [22]:
test_fe.head()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin
0,0,0,1,8,1,0,0,1,0,0,...,9,1,1,1,12,0,1,1,0,0
1,1,4,2,5,1,0,0,0,0,1,...,7,2,0,3,10,0,0,1,1,0
2,2,5,1,3,0,0,0,0,0,1,...,12,4,0,2,4,0,0,0,0,0
3,3,0,1,6,0,0,1,0,0,0,...,13,5,1,0,5,1,0,1,0,0
4,4,5,1,7,0,0,0,0,0,1,...,12,4,0,0,4,0,1,1,0,0


In [23]:
train_train_x_fe= train_train_x.drop(['ps_ind_13_bin','ps_ind_12_bin', 'ps_ind_11_bin','ps_ind_10_bin','ps_calc_20_bin'],axis=1)
train_test_x_fe= train_test_x.drop(['ps_ind_13_bin','ps_ind_12_bin', 'ps_ind_11_bin','ps_ind_10_bin','ps_calc_20_bin'],axis=1)
train_x_fe= train_x_fe.drop(['ps_ind_13_bin','ps_ind_12_bin', 'ps_ind_11_bin','ps_ind_10_bin','ps_calc_20_bin'],axis=1)

In [24]:
#binary sums
test_fe['binary_sums_17_16_7_6']=test_fe['ps_ind_17_bin']+test_fe['ps_ind_16_bin']+test_fe['ps_ind_07_bin']+test_fe['ps_ind_06_bin']
train_test_x_fe['binary_sums_17_16_7_6']=train_test_x_fe['ps_ind_17_bin']+train_test_x_fe['ps_ind_16_bin']+train_test_x_fe['ps_ind_07_bin']+train_test_x_fe['ps_ind_06_bin']
train_train_x_fe['binary_sums_17_16_7_6']=train_train_x_fe['ps_ind_17_bin']+train_train_x_fe['ps_ind_16_bin']+train_train_x_fe['ps_ind_07_bin']+train_train_x_fe['ps_ind_06_bin']
train_x_fe['binary_sums_17_16_7_6']=train_x_fe['ps_ind_17_bin']+train_x_fe['ps_ind_16_bin']+train_x_fe['ps_ind_07_bin']+train_x_fe['ps_ind_06_bin']

In [25]:
#car_13, reg03 sum
test_fe['sum_car_13_reg03']= test_fe['ps_car_13']+test_fe['ps_reg_03']
train_train_x_fe['sum_car_13_reg03']= train_train_x_fe['ps_car_13']+train_train_x_fe['ps_reg_03']
train_test_x_fe['sum_car_13_reg03']= train_test_x_fe['ps_car_13']+train_test_x_fe['ps_reg_03']
train_x_fe['sum_car_13_reg03']= train_x_fe['ps_car_13']+train_x_fe['ps_reg_03']


In [26]:

#car_13, reg03 mult
test_fe['mult_car_13_reg03']= test_fe['ps_car_13']*test_fe['ps_reg_03']
train_train_x_fe['mult_car_13_reg03']= train_train_x_fe['ps_car_13']*train_train_x_fe['ps_reg_03']
train_test_x_fe['mult_car_13_reg03']= train_test_x_fe['ps_car_13']*train_test_x_fe['ps_reg_03']
train_x_fe['mult_car_13_reg03']= train_x_fe['ps_car_13']*train_x_fe['ps_reg_03']


In [27]:
#car_13 sq
test_fe['sq_car_13']= test_fe['ps_car_13']*test_fe['ps_car_13']
train_train_x_fe['sq_car_13']= train_train_x_fe['ps_car_13']*train_train_x_fe['ps_car_13']
train_test_x_fe['sq_car_13']= train_test_x_fe['ps_car_13']*train_test_x_fe['ps_car_13']
train_x_fe['sq_car_13']= train_x_fe['ps_car_13']*train_x_fe['ps_car_13']


In [28]:
#reg03 sq
test_fe['sq_reg03']= test_fe['ps_car_13']*test_fe['ps_reg_03']
train_train_x_fe['sq_reg03']= train_train_x_fe['ps_reg_03']*train_train_x_fe['ps_reg_03']
train_test_x_fe['sq_reg03']= train_test_x_fe['ps_reg_03']*train_test_x_fe['ps_reg_03']
train_x_fe['sq_reg03']= train_x_fe['ps_reg_03']*train_x_fe['ps_reg_03']


In [29]:
#car 14, car 12 , ind 15 sum
test_fe['sum_car14_car12_ind15']= test_fe['ps_car_14']+test_fe['ps_car_12']+test_fe['ps_ind_15']
train_test_x_fe['sum_car14_car12_ind15']= train_test_x_fe['ps_car_14']+train_test_x_fe['ps_car_12']+train_test_x_fe['ps_ind_15']
train_train_x_fe['sum_car14_car12_ind15']= train_train_x_fe['ps_car_14']+train_train_x_fe['ps_car_12']+train_train_x_fe['ps_ind_15']
train_x_fe['sum_car14_car12_ind15']= train_x_fe['ps_car_14']+train_x_fe['ps_car_12']+train_x_fe['ps_ind_15']

In [30]:
#car 14, car 12 , ind 15 mult
test_fe['mult_car14_car12_ind15']= test_fe['ps_car_14']*test_fe['ps_car_12']*test_fe['ps_ind_15']
train_test_x_fe['mult_car14_car12_ind15']= train_test_x_fe['ps_car_14']*train_test_x_fe['ps_car_12']*train_test_x_fe['ps_ind_15']
train_train_x_fe['mult_car14_car12_ind15']= train_train_x_fe['ps_car_14']*train_train_x_fe['ps_car_12']*train_train_x_fe['ps_ind_15']
train_x_fe['mult_car14_car12_ind15']= train_x_fe['ps_car_14']*train_x_fe['ps_car_12']*train_x_fe['ps_ind_15']

# What's next:
The project bifurcates here. 
The final 'production' result is going to be an absurd ensemble:
https://www.kaggle.com/zusmani/lgb-esemble-xgb-be-in-top-100-with-lb-0-285/output
has acceptably clean code for this. 
So I can save out my train and test feature engineering and run that code. 

The rest of this notebook is for hyperparameter optimization for each of the component models. 

step 5: Xgboost hyperparameter tuning
https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
explain what I'm doing here, in short, lets find the best hyperparameters for xgboost


In [31]:
test_fe.columns

Index(['id', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'binary_sums_17_16_7_6', 'sum_car_13_reg03', 'mult_car_13_reg03',
       'sq_car_13', 'sq_reg03', 'sum_c

In [32]:
train_train_x.columns

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

In [33]:
# Define the gini metric - from https://www.kaggle.com/c/ClaimPredictionChallenge/discussion/703#5897
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

# Create an XGBoost-compatible metric from Gini

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return 'gini', gini_score
    

xgbscores = []

# Set xgb parameters

params = {}
params['objective'] = 'binary:logistic'
params['eta'] = 0.03
params['silent'] = True
params['max_depth'] = 8
params['subsample'] = 0.9
params['colsample_bytree'] = 0.85
params['colsample_bylevel'] = 0.9
#params['tree_method'] = 'exact'






The first step is to find how many trees. 
I'm not going to use crossval, simply due to computational expense. 

In [34]:
params = {}
params['objective'] = 'binary:logistic'
params['eta'] = 0.025
params['silent'] = True
params['max_depth'] = 8
params['subsample'] = 0.9
params['colsample_bytree'] = 0.85
params['colsample_bylevel'] = 0.9
d_train = xgb.DMatrix(train_train_x_fe, train_train_y)
d_valid = xgb.DMatrix(train_test_x_fe, train_test_y)
d_final_test = xgb.DMatrix(train_x_fe, label=train_y)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
mdl = xgb.train(params, d_train, 1600, watchlist, early_stopping_rounds=70, feval=gini_xgb, maximize=True, verbose_eval=100)

[0]	train-gini:0.230157	valid-gini:0.214863
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 70 rounds.
[100]	train-gini:0.362429	valid-gini:0.261215
[200]	train-gini:0.481147	valid-gini:0.27294
[300]	train-gini:0.57091	valid-gini:0.276014
Stopping. Best iteration:
[306]	train-gini:0.575042	valid-gini:0.276217



In [35]:
gini_scorer = metrics.make_scorer(gini_normalized, greater_is_better = True)

In [36]:
from sklearn.metrics import roc_auc_score

In [37]:

param_test1 = {}
param_test1['max_depth']=[3,5,6,8,10]
param_test1['min_child_weight']=[.5,.75,2]

param_test1['objective'] = ['binary:logistic']
#param_test1['eta'] = [0.03]
param_test1['silent'] = [True]
param_test1['subsample'] = [.8]
param_test1['colsample_bytree'] = [0.85]
param_test1['colsample_bylevel'] = [0.9]
param_test1['scale_pos_weight'] = [1.6]
param_test1['n_estimators']=[306]
param_test1['learning_rate']=[.025]

gsearch1 = GridSearchCV(estimator = XGBClassifier(n_estimators=322,nthread=-1), param_grid = param_test1,iid=False,scoring = gini_scorer, cv=5)
gsearch1.fit(train_x_fe,train_y)


KeyboardInterrupt: 

In [ ]:

param_test1 = {}
param_test1['max_depth']=[6,8,10]
param_test1['min_child_weight']=[.5,2]

param_test1['objective'] = ['binary:logistic']
#param_test1['eta'] = [0.03]
param_test1['silent'] = [True]
param_test1['subsample'] = [.8]
param_test1['colsample_bytree'] = [0.85]
param_test1['colsample_bylevel'] = [0.9]
param_test1['scale_pos_weight'] = [1.6]
param_test1['n_estimators']=[306]
param_test1['learning_rate']=[.025]

gsearch1 = GridSearchCV(estimator = XGBClassifier(n_estimators=322,nthread=-1), param_grid = param_test1,iid=False,scoring = 'roc_auc', cv=3)
gsearch1.fit(train_x_fe,train_y)


In [41]:
gsearch1.grid_scores_

[mean: 0.63883, std: 0.00084, params: {'objective': 'binary:logistic', 'scale_pos_weight': 1.6, 'colsample_bylevel': 0.9, 'silent': True, 'min_child_weight': 0.5, 'learning_rate': 0.025, 'max_depth': 6, 'n_estimators': 306, 'subsample': 0.8, 'colsample_bytree': 0.85},
 mean: 0.63903, std: 0.00071, params: {'objective': 'binary:logistic', 'scale_pos_weight': 1.6, 'colsample_bylevel': 0.9, 'silent': True, 'min_child_weight': 2, 'learning_rate': 0.025, 'max_depth': 6, 'n_estimators': 306, 'subsample': 0.8, 'colsample_bytree': 0.85},
 mean: 0.63692, std: 0.00072, params: {'objective': 'binary:logistic', 'scale_pos_weight': 1.6, 'colsample_bylevel': 0.9, 'silent': True, 'min_child_weight': 0.5, 'learning_rate': 0.025, 'max_depth': 8, 'n_estimators': 306, 'subsample': 0.8, 'colsample_bytree': 0.85},
 mean: 0.63717, std: 0.00097, params: {'objective': 'binary:logistic', 'scale_pos_weight': 1.6, 'colsample_bylevel': 0.9, 'silent': True, 'min_child_weight': 2, 'learning_rate': 0.025, 'max_depth

In [42]:
gsearch1.best_params_

{'colsample_bylevel': 0.9,
 'colsample_bytree': 0.85,
 'learning_rate': 0.025,
 'max_depth': 6,
 'min_child_weight': 2,
 'n_estimators': 306,
 'objective': 'binary:logistic',
 'scale_pos_weight': 1.6,
 'silent': True,
 'subsample': 0.8}

In [43]:
gsearch1.best_score_

0.6390312496524134

In [ ]:
roc_auc_score(gsearch1.predict(train_x_fe),train_y)

Step 6: Catboost+Parameter Tuning


In [44]:
cat_model = CatBoostClassifier()

In [45]:
cat_early = CatBoostClassifier(iterations=1200, learning_rate=0.03, depth=8, loss_function='Logloss', eval_metric='AUC', random_seed=99, od_type='Iter', od_wait=100) 
cat_early.fit(train_train_x_fe, train_train_y, eval_set=(train_test_x_fe,train_test_y), use_best_model=True, verbose=True)

Borders for float features generated
0:	learn 0.5328927227	test 0.5305296051	bestTest 0.5305296051		total: 661ms	remaining: 13m 12s
1:	learn 0.5913348968	test 0.5887532093	bestTest 0.5887532093		total: 1.29s	remaining: 12m 53s
2:	learn 0.5932298814	test 0.5902804054	bestTest 0.5902804054		total: 1.91s	remaining: 12m 40s
3:	learn 0.596211205	test 0.5928300364	bestTest 0.5928300364		total: 2.54s	remaining: 12m 38s
4:	learn 0.5973418459	test 0.5933124006	bestTest 0.5933124006		total: 3.14s	remaining: 12m 31s
5:	learn 0.5991870177	test 0.5941255068	bestTest 0.5941255068		total: 3.84s	remaining: 12m 45s
6:	learn 0.598259185	test 0.5914457123	bestTest 0.5941255068		total: 4.5s	remaining: 12m 47s
7:	learn 0.6009381574	test 0.5926909809	bestTest 0.5941255068		total: 5.18s	remaining: 12m 51s
8:	learn 0.6014148894	test 0.5943446144	bestTest 0.5943446144		total: 5.83s	remaining: 12m 51s
9:	learn 0.6046120299	test 0.5984664796	bestTest 0.5984664796		total: 6.54s	remaining: 12m 57s
10:	learn 0.6065

86:	learn 0.6366608634	test 0.6285884542	bestTest 0.6285884542		total: 57.9s	remaining: 12m 20s
87:	learn 0.6367442068	test 0.6287441904	bestTest 0.6287441904		total: 58.6s	remaining: 12m 20s
88:	learn 0.6369154836	test 0.6288335774	bestTest 0.6288335774		total: 59.3s	remaining: 12m 20s
89:	learn 0.6370903664	test 0.6290878588	bestTest 0.6290878588		total: 1m	remaining: 12m 20s
90:	learn 0.6371688537	test 0.629277955	bestTest 0.629277955		total: 1m	remaining: 12m 19s
91:	learn 0.6372514619	test 0.6291631731	bestTest 0.629277955		total: 1m 1s	remaining: 12m 18s
92:	learn 0.6371566616	test 0.629167487	bestTest 0.629277955		total: 1m 1s	remaining: 12m 17s
93:	learn 0.6371944492	test 0.6293040229	bestTest 0.6293040229		total: 1m 2s	remaining: 12m 16s
94:	learn 0.6371603828	test 0.6293693874	bestTest 0.6293693874		total: 1m 3s	remaining: 12m 15s
95:	learn 0.6370973484	test 0.6293801369	bestTest 0.6293801369		total: 1m 3s	remaining: 12m 14s
96:	learn 0.6371627008	test 0.6294244828	bestTest 0

171:	learn 0.6455650839	test 0.6343701183	bestTest 0.6343701183		total: 1m 55s	remaining: 11m 29s
172:	learn 0.6456143334	test 0.6344028849	bestTest 0.6344028849		total: 1m 56s	remaining: 11m 29s
173:	learn 0.6456838104	test 0.6344519646	bestTest 0.6344519646		total: 1m 56s	remaining: 11m 28s
174:	learn 0.6458081238	test 0.6345712874	bestTest 0.6345712874		total: 1m 57s	remaining: 11m 28s
175:	learn 0.6459204694	test 0.6345549141	bestTest 0.6345712874		total: 1m 58s	remaining: 11m 27s
176:	learn 0.64595147	test 0.6345679078	bestTest 0.6345712874		total: 1m 58s	remaining: 11m 26s
177:	learn 0.6460553528	test 0.634645788	bestTest 0.634645788		total: 1m 59s	remaining: 11m 26s
178:	learn 0.6462119467	test 0.634738874	bestTest 0.634738874		total: 2m	remaining: 11m 25s
179:	learn 0.646401705	test 0.6347642206	bestTest 0.6347642206		total: 2m	remaining: 11m 24s
180:	learn 0.6465034262	test 0.6347760229	bestTest 0.6347760229		total: 2m 1s	remaining: 11m 23s
181:	learn 0.6466631453	test 0.63479

256:	learn 0.6545638302	test 0.6366717045	bestTest 0.6366717045		total: 3m	remaining: 11m
257:	learn 0.6547229876	test 0.6366703242	bestTest 0.6366717045		total: 3m	remaining: 10m 59s
258:	learn 0.6547917753	test 0.6366590403	bestTest 0.6366717045		total: 3m 1s	remaining: 10m 59s
259:	learn 0.6549925764	test 0.6367013147	bestTest 0.6367013147		total: 3m 2s	remaining: 10m 58s
260:	learn 0.6550437625	test 0.6367237177	bestTest 0.6367237177		total: 3m 2s	remaining: 10m 57s
261:	learn 0.6550896881	test 0.6367352387	bestTest 0.6367352387		total: 3m 3s	remaining: 10m 56s
262:	learn 0.6552467038	test 0.6367678868	bestTest 0.6367678868		total: 3m 4s	remaining: 10m 55s
263:	learn 0.6553554784	test 0.6368049372	bestTest 0.6368049372		total: 3m 4s	remaining: 10m 54s
264:	learn 0.6553794551	test 0.6368030345	bestTest 0.6368049372		total: 3m 5s	remaining: 10m 54s
265:	learn 0.655519623	test 0.636808369	bestTest 0.636808369		total: 3m 6s	remaining: 10m 53s
266:	learn 0.6556512119	test 0.6368286221	b

341:	learn 0.6640959427	test 0.6385692467	bestTest 0.6386094355		total: 3m 58s	remaining: 9m 57s
342:	learn 0.6641635293	test 0.6385761887	bestTest 0.6386094355		total: 3m 58s	remaining: 9m 56s
343:	learn 0.6643249115	test 0.6386452121	bestTest 0.6386452121		total: 3m 59s	remaining: 9m 56s
344:	learn 0.6643790523	test 0.638660844	bestTest 0.638660844		total: 4m	remaining: 9m 55s
345:	learn 0.664447999	test 0.6386896002	bestTest 0.6386896002		total: 4m	remaining: 9m 54s
346:	learn 0.6645080638	test 0.6386851779	bestTest 0.6386896002		total: 4m 1s	remaining: 9m 53s
347:	learn 0.6645675857	test 0.6387419108	bestTest 0.6387419108		total: 4m 2s	remaining: 9m 53s
348:	learn 0.6646140062	test 0.6387671348	bestTest 0.6387671348		total: 4m 2s	remaining: 9m 52s
349:	learn 0.6648024202	test 0.6387656982	bestTest 0.6387671348		total: 4m 3s	remaining: 9m 51s
350:	learn 0.6649143719	test 0.6387840948	bestTest 0.6387840948		total: 4m 4s	remaining: 9m 50s
351:	learn 0.6649518853	test 0.6388032187	best

427:	learn 0.6711946461	test 0.6393528181	bestTest 0.6393530412		total: 4m 57s	remaining: 8m 56s
428:	learn 0.6712513038	test 0.639378448	bestTest 0.639378448		total: 4m 58s	remaining: 8m 56s
429:	learn 0.671336326	test 0.6393115363	bestTest 0.639378448		total: 4m 59s	remaining: 8m 55s
430:	learn 0.6713851614	test 0.6393053117	bestTest 0.639378448		total: 4m 59s	remaining: 8m 54s
431:	learn 0.6714867574	test 0.6393455246	bestTest 0.639378448		total: 5m	remaining: 8m 54s
432:	learn 0.6715987701	test 0.6393084401	bestTest 0.639378448		total: 5m 1s	remaining: 8m 53s
433:	learn 0.6716481948	test 0.6393119602	bestTest 0.639378448		total: 5m 1s	remaining: 8m 52s
434:	learn 0.671723352	test 0.6392972606	bestTest 0.639378448		total: 5m 2s	remaining: 8m 51s
435:	learn 0.6718440958	test 0.6392370558	bestTest 0.639378448		total: 5m 3s	remaining: 8m 51s
436:	learn 0.6718896318	test 0.6392372868	bestTest 0.639378448		total: 5m 3s	remaining: 8m 50s
437:	learn 0.6719818602	test 0.6392619704	bestTest 

513:	learn 0.6779818609	test 0.6392601058	bestTest 0.639378448		total: 5m 55s	remaining: 7m 54s
514:	learn 0.6780653937	test 0.6392253439	bestTest 0.639378448		total: 5m 56s	remaining: 7m 53s
515:	learn 0.67818244	test 0.6392192861	bestTest 0.639378448		total: 5m 56s	remaining: 7m 53s
516:	learn 0.678210276	test 0.639221239	bestTest 0.639378448		total: 5m 57s	remaining: 7m 52s
517:	learn 0.6782975176	test 0.639197522	bestTest 0.639378448		total: 5m 58s	remaining: 7m 51s
518:	learn 0.6783787974	test 0.6391920026	bestTest 0.639378448		total: 5m 58s	remaining: 7m 50s
519:	learn 0.6784224238	test 0.6391788742	bestTest 0.639378448		total: 5m 59s	remaining: 7m 50s
520:	learn 0.6784316104	test 0.639190789	bestTest 0.639378448		total: 6m	remaining: 7m 49s
521:	learn 0.6784544653	test 0.6391764993	bestTest 0.639378448		total: 6m	remaining: 7m 48s
522:	learn 0.6785614715	test 0.6391635116	bestTest 0.639378448		total: 6m 1s	remaining: 7m 47s
523:	learn 0.6786830838	test 0.6391646629	bestTest 0.63

In [46]:
cat_early_pred = cat_early.predict(train_test_x_fe)

In [47]:
cat_early_pred

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [48]:
train_test_y

191093    0
409985    0
54034     0
400856    0
469504    0
184907    0
180428    0
153828    0
4289      0
460504    0
470843    0
174608    0
52734     0
499303    0
420582    0
388995    0
189185    0
247608    0
481146    0
588497    0
47016     1
130316    0
538984    0
299586    0
271864    0
590495    0
112410    0
209711    0
112610    0
360230    0
         ..
256082    0
254452    0
562931    0
542760    0
393656    0
179668    0
126631    0
155724    0
88154     0
503233    0
499176    0
8512      0
22514     0
533326    0
93960     0
91304     0
382114    0
197514    0
141291    0
105344    0
146730    0
153157    0
292372    0
195660    0
187126    0
49304     0
97290     0
505638    0
284309    0
124773    0
Name: target, Length: 119043, dtype: int64

In [49]:
# CatBoost params
cat_params1 = {}
cat_params1['iterations'] = [429]
cat_params1['depth'] = [8,6,10]
cat_params1['rsm'] = [0.95]
cat_params1['learning_rate'] = [0.03,.02]
cat_params1['l2_leaf_reg'] = [3.5,2]
cat_params1['border_count'] = [8,7,6]
cat_params1['gradient_iterations'] = [5,4,3]


In [52]:
gsearch2 = GridSearchCV(estimator = CatBoostClassifier(), param_grid = cat_params1,iid=False,scoring = 'roc_auc', cv=3, n_jobs=-1)
gsearch2.fit(train_x_fe,train_y)

ImportError: [joblib] Attempting to do parallel computing without protecting your import on a system that does not support forking. To use parallel-computing in a script, you must protect your main loop using "if __name__ == '__main__'". Please see the joblib documentation on Parallel for more information

In [ ]:
gsearch2.best_params_

In [ ]:
gsearch2.best_score_

In [ ]:
test_fe.columns

In [ ]:
train_test_x_fe.columns

In [53]:
test_fe.shape

(892816, 60)

Step 7: Lightgbm + parameter tuning

In [ ]:
d_train = lgb.Dataset(train_train_x, train_train_y)
d_valid = lgb.Dataset(train_test_x,train_test_y) 

watchlist = [d_train, d_valid]


print('Training LGBM model...')
lgb_params = {}
lgb_params['learning_rate'] = 0.02
lgb_params['application'] = 'binary'
lgb_params['max_depth'] = 14
lgb_params['num_leaves'] = 2**8
lgb_params['verbosity'] = 0
lgb_params['metric'] = 'auc'
lgb_params['max_bin'] = 10
lgb_params['subsample'] = 0.8
lgb_params['subsample_freq'] = 10
lgb_params['colsample_bytree'] = 0.8   
lgb_params['min_child_samples'] = 500

model = lgb.train(params, train_set=d_train, num_boost_round=1200, valid_sets=watchlist, \
early_stopping_rounds=25, verbose_eval=10)


step 8: LR stack